In [15]:
import tables
import numpy as np
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

In [16]:
hdf5_path = '/Users/chetan/Documents/Git Projects/Machine Learning Projects/Computer Vision Problem/Dataset/dataset.hdf5'  # path of the dataset.hdf5 file
subtract_mean = True
batch_size = 50
nb_class = 2

In [17]:
hdf5_file = tables.open_file(hdf5_path, mode='r')
# subtract the training mean
if subtract_mean:
    mm = hdf5_file.root.train_mean[0]
    mm = mm[np.newaxis, ...]

# Total number of samples
train_data = np.array(hdf5_file.root.train_img)
train_label = np.array(hdf5_file.root.train_labels)

test_data = np.array(hdf5_file.root.test_img)
test_label = np.array(hdf5_file.root.test_labels)

val_data = np.array(hdf5_file.root.val_img)
val_label = np.array(hdf5_file.root.val_labels)

print('train data:',train_data.shape,' train_label',train_label.shape)
print('test_data:',test_data.shape,' test_label:',test_label.shape)
print('val_data:',val_data.shape,' val_label:',val_label.shape)

train data: (259, 224, 224, 3)  train_label (259,)
test_data: (87, 224, 224, 3)  test_label: (87,)
val_data: (86, 224, 224, 3)  val_label: (86,)


In [18]:
from keras.utils import np_utils

# one-hot encode the labels
num_classes = len(np.unique(train_label))
train_label = np_utils.to_categorical(train_label, num_classes)
test_label = np_utils.to_categorical(test_label, num_classes)
val_label = np_utils.to_categorical(val_label, num_classes)

# print shape of training set
print('num_classes:', num_classes)

# print number of training, validation, and test images
print(train_label.shape, 'train samples')
print(test_label.shape, 'test samples')
print(val_label.shape, 'validation samples')

num_classes: 2
(259, 2) train samples
(87, 2) test samples
(86, 2) validation samples


In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 64)        0         
__________

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [21]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
hist = model.fit(train_data, train_label, batch_size=None, epochs=20,
          validation_data=(val_data, val_label),callbacks=[checkpointer], 
          verbose=1, shuffle=True)

Coming here
Train on 259 samples, validate on 86 samples
Epoch 1/20
259/259 [==============================] - 18s 70ms/step - loss: 5.1563 - acc: 0.6178 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00001: val_loss improved from inf to 5.24775, saving model to model.weights.best.hdf5
Epoch 2/20
259/259 [==============================] - 13s 52ms/step - loss: 5.6631 - acc: 0.6486 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00002: val_loss did not improve
Epoch 3/20
259/259 [==============================] - 14s 53ms/step - loss: 5.6631 - acc: 0.6486 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00003: val_loss did not improve
Epoch 4/20
259/259 [==============================] - 13s 51ms/step - loss: 5.6631 - acc: 0.6486 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00004: val_loss did not improve
Epoch 5/20
259/259 [==============================] - 13s 52ms/step - loss: 5.6631 - acc: 0.6486 - val_loss: 5.2478 - val_acc: 0.6744

Epoch 00005: val_loss did not improve
Epoch 6/20
259/259 [====

In [22]:
model.load_weights('model.weights.best.hdf5')

In [23]:
score = model.evaluate(test_data, test_label, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.6551724100249937
